In [1]:
import numpy as np
from numpy import radians, pi
from lib.cartpolesystem import CartPoleSystem, Cart, Pole, StepperMotor
from lib.cartpoleenv import CartPoleEnv
from lib.regulators import LQR
import time
from time import perf_counter
from lib.numerical import rk4_step

In [2]:
dt = 0.005
g = 9.81
r = 0.04456
J = 0.0001
d = 0.0001

cart = Cart(0.2167, 0, (-0.8, 0.8), 0.1)
motor = StepperMotor(r, (-10, 10), 0.2, (-2, 2), 0.1)
poles = [Pole(0.045, 0.15, 0.075, d, J)]
n = len(poles)

system = CartPoleSystem(cart, motor, poles, g)

max_time = 10
N = int(max_time/dt)

env = CartPoleEnv(system, dt, rk4_step)
env.observation_space.shape

Checking if matlab equations exist...
Using matlab equations
Setting casadi equations
sp_vars [s, d_s, theta1, d_theta1, dd_s, dd_theta1]
sp_sols [(8.54296875e-7*d_theta1**2*sin(theta1)**3 + 8.54296875e-7*d_theta1**2*sin(theta1)*cos(theta1)**2 + 3.375e-7*d_theta1**2*sin(theta1) + 3.375e-7*d_theta1*cos(theta1) + 6.62428125e-5*dd_s*sin(theta1)**2 + 5.48521875e-5*dd_s*cos(theta1)**2 + 2.617e-5*dd_s - 0.00011174203125*sin(theta1)*cos(theta1))/(0.000253125*sin(theta1)**2 + 0.000253125*cos(theta1)**2 + 0.0001), (-0.0001*d_theta1 + 0.003375*dd_s*cos(theta1) + 0.03310875*sin(theta1))/(0.000253125*sin(theta1)**2 + 0.000253125*cos(theta1)**2 + 0.0001)]


(4,)

In [3]:
linearization_x0 = np.array([0, 0] + [radians(0), 0]*n)
linearization_u0 = np.array([0])

A0, B0 = system.linearize(linearization_x0, linearization_u0)

C = np.eye(2+2*n)
D = np.zeros((2+2*n, 1))

ct_dt = dt

Q = np.diag([100, 10] + [100, 10]*n)
R = np.diag([1])

A_d0, B_d0 = LQR.discretize(ct_dt, A0, B0, C, D)
_, K_d0 = LQR.calculate_K_d(A_d0, B_d0, Q, R)

In [4]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print("A")
print(A0)
print("B")
print(B0)

A
[[ 0.000  1.000  0.000  0.000]
 [ 0.000  0.000  0.000  0.000]
 [ 0.000  0.000  0.000  1.000]
 [ 0.000  0.000 -0.316  0.001]]
B
[[ 0.000]
 [ 1.000]
 [ 0.000]
 [ 0.229]]


In [5]:
def is_controllable(A, B):
    n = A.shape[0]  # Number of states
    controllability_matrix = np.column_stack([np.linalg.matrix_power(A, i) @ B for i in range(n)])
    rank = np.linalg.matrix_rank(controllability_matrix)
    return rank == n

if is_controllable(A0, B0):
    print("The system is controllable.")
else:
    print("The system is not controllable.")

The system is controllable.


In [6]:
x0 = np.array([0, 0, radians(-1), 0, radians(0), 0])

state, _ = env.reset(x0)

r = np.array([0.3, 0] + [radians(0), 0]*n)

last_update = perf_counter()

time1 = time2 = 0

for i in range(N-1):
    while perf_counter() < last_update + dt:
        pass
    last_update = perf_counter()
    error = system.calculate_error(state, r)
    control = LQR.feedback(K_d0, error)
    time1 = time.perf_counter()
    state, reward, done, msg, _ = env.step(control)
    time2 = time.perf_counter()
    env.render()
env.close()
print(time2-time1)

RuntimeError: Error in Function::call for 'constraint_states' [SXFunction] at .../casadi/core/function.cpp:330:
.../casadi/core/function_internal.hpp:1618: Assertion "arg.size()==n_in_" failed:
Incorrect number of inputs: Expected 5, got 7